In [61]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [62]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "pesel",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [63]:
employees_df

,id,imię,nazwisko,pesel,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Magdalena,Falkowska,90080511206,Jarocin,ul. Dębowa 25/15,48931843254,magdalena.falkowska@kuba.pl,2014-01-01,None,menadżer,62.0
1,2,Lucjan,Kramarz,86112107178,Bartoszyce,ul. Pograniczników 94/49,48128706704,lucjan.kramarz@kuba.pl,2014-01-01,2016-04-14,sprzedawca,58.4
2,3,Marta,Burchacka,61042051184,Wrocław,ul. Marii Konopnickiej 68,48339241328,marta.burchacka@kuba.pl,2014-01-01,None,sprzedawca,52.1
4,4,Halina,Feluś,82100971226,Elbląg,ul. Budzówka 8/29,48359495725,halina.felus@kuba.pl,2014-01-01,2015-08-16,mechanik,48.4
11,5,Wioletta,Okroj,89083106263,Rzepin,ul. Mazura 47,48358487814,wioletta.okroj@kuba.pl,2014-01-01,None,informatyk,47.5
3,6,Wojciech,Safronov,74081384557,Wrocław,ul. Osiedlowa 22/38,48905707614,wojciech.safronov@kuba.pl,2014-03-19,None,sprzedawca,58.0
5,7,Yan,Hura,95061852790,Wrocław,ul. Mahoniowa 80/36,48209699419,yan.hura@kuba.pl,2015-03-28,None,mechanik,55.3
6,8,Michał,Walasek,75040677354,Wrocław,ul. Modrzewiowa 26/40,48041822788,michal.walasek@kuba.pl,2015-09-01,None,mechanik,55.0
7,9,Dariusz,Peroń,89091874394,Wrocław,ul. Piaskowa 85/49,48429216030,dariusz.peron@kuba.pl,2016-02-26,None,pomocnik mechanika,42.9
8,10,Zofia,Puławska,76030669403,Wrocław,ul. Leśna 4/15,48590130563,zofia.pulawska@kuba.pl,2016-04-28,None,diagnosta samochodowy,52.4


In [64]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [65]:
no_transactions = 2000
begining_buy = 20

dates = random_transaction_dates("2014-01-01", "2020-12-31", no_transactions)

In [66]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [67]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(5*max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            jp += 1
            jc += 1

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            jp += 1
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1

In [68]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [69]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [70]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [71]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [72]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [73]:
year_lims = [1970, 2010]
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])
unique_clients = np.unique(client_ids)
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man(year_lims)
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_lims[0], year_lims[1]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man(year_lims)
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_lims[0], year_lims[1]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Stefaniia,Murashkova,K,97110396534,Wrocław,ul. Tęczowa 85/29,False,48709301163,s-mur1997@gmail.com
1,2,Dariusz,Czarnolewski,M,86053013316,Wrocław,ul. Świerkowa 22,False,48561145823,czarnolewski19866@gmail.com
2,3,Urszula,Zagrajek,K,80103135713,Małogoszcz,ul. Pomorska 69/47,True,48004768068,u.zagrajek80@wp.pl
3,4,Alan,Zhyhalskyi,M,72111160173,Wrocław,ul. Sudel 61/7,False,48128707860,qxo8h8@opayq.com
4,5,Wiesława,Falarz,K,91042511832,Kraków,ul. św. Wiktorii 3/42,False,48046902102,um7lcutn662xf@opayq.com
...,...,...,...,...,...,...,...,...,...,...
1464,1465,Nikola,Brzozowska,K,81071649411,Kraków,ul. Jana Brzechwy 84/2,False,48031187355,brzozowsk-nik1981@vp.pl
1465,1466,Katarzyna,Moskal,K,94121213256,Wrocław,ul. Cicha 41/23,False,48667154802,m_katar1994@interia.pl
1466,1467,Dariusz,Cieślak,M,00240947651,Świebodzin,ul. Gęstych Domów 97,False,48635418253,d.c2770@wp.pl
1467,1468,Agnieszka,Rypianchyn,K,09260341399,Ozimek,ul. Cicha 29/44,False,48837001135,kc10bjjf6zw6c03@opayq.com


In [74]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Stefaniia,Murashkova,K,97110396534,Wrocław,ul. Tęczowa 85/29,False,48709301163,s-mur1997@gmail.com
1,2,Dariusz,Czarnolewski,M,86053013316,Wrocław,ul. Świerkowa 22,False,48561145823,czarnolewski19866@gmail.com
2,3,Urszula,Zagrajek,K,80103135713,Małogoszcz,ul. Pomorska 69/47,True,48004768068,u.zagrajek80@wp.pl
3,4,Alan,Zhyhalskyi,M,72111160173,Wrocław,ul. Sudel 61/7,False,48128707860,qxo8h8@opayq.com
4,5,Wiesława,Falarz,K,91042511832,Kraków,ul. św. Wiktorii 3/42,False,48046902102,um7lcutn662xf@opayq.com
...,...,...,...,...,...,...,...,...,...,...
1464,1465,Nikola,Brzozowska,K,81071649411,Kraków,ul. Jana Brzechwy 84/2,False,48031187355,brzozowsk-nik1981@vp.pl
1465,1466,Katarzyna,Moskal,K,94121213256,Wrocław,ul. Cicha 41/23,False,48667154802,m_katar1994@interia.pl
1466,1467,Dariusz,Cieślak,M,00240947651,Świebodzin,ul. Gęstych Domów 97,False,48635418253,d.c2770@wp.pl
1467,1468,Agnieszka,Rypianchyn,K,09260341399,Ozimek,ul. Cicha 29/44,False,48837001135,kc10bjjf6zw6c03@opayq.com


In [75]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [76]:
np.sum(clients_df['karta_lojalnościowa'])

np.int64(88)

In [77]:
np.sum(pd.isna(clients_df["PESEL"]))

np.int64(55)

# auta

In [78]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [79]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [80]:
df_samochody = create_car_table(max(car_ids))

In [81]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Mercedes-Benz,E 350,4,2000,SUV,False,biały,Diesel,4.2,258,Automatyczna,5,5,False
1,2,Skoda,Fabia,4,2010,Hatchback,False,szary,Benzyna,1.3,90,Manualna,5,5,False
2,3,Bajaj,Discover 125 ST,2,2016,,False,czarny,Benzyna,0.8,109,Automatyczna,0,1,False
3,4,Volkswagen,T-Roc,4,2004,SUV,True,niebieski,Benzyna,2.4,150,Automatyczna,3,2,False
4,5,Royal,Enfield Thunder 500,2,2005,,True,szary,Benzyna,0.3,46,Automatyczna,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,968,Ford,EcoSport,4,2019,Sedan,True,szary,Benzyna,2.0,125,Manualna,3,5,False
968,969,SEAT,Leon,4,2008,SUV,False,czarny,Benzyna,3.4,204,Manualna,5,5,False
969,970,Ford,Mondeo,4,2002,Coupe,False,biały,Benzyna,2.4,150,Manualna,5,2,False
970,971,Skoda,Fabia,4,2010,Sedan,False,czarny,Benzyna,1.1,69,Automatyczna,5,5,True


In [82]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [83]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Automatyczna', 'Manualna', 'Półautomatyczna'], dtype=object)

# Usługi

In [84]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [85]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [86]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [87]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_10328\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2006' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [88]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [89]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,2,2014-01-02,71800.0
1,2,2,2,6,2014-01-04,46200.0
2,3,3,3,3,2014-01-04,28300.0
3,4,4,4,2,2014-01-08,36600.0
4,5,5,5,6,2014-01-11,9500.0
...,...,...,...,...,...,...
934,935,969,1467,6,2020-12-12,67200.0
935,936,970,558,6,2020-12-13,29700.0
936,937,971,89,3,2020-12-26,18500.0
937,938,729,1165,3,2020-12-27,48900.0


In [90]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [91]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [99]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

for i in range(len(sell_ids)):
    customer_id = df_sell.loc()[i]['id_klienta']
    if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool():
        sell_prices[i] *= 0.97

df_sell['cena'] = sell_prices

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_10328\2566026374.py:8: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool():


58900.0
57133.0
42300.0
41031.0
29200.0
28324.0
110700.0
107379.0
55000.0
53350.0
52100.0
50537.0
28200.0
27354.0
59900.0
58103.0
38500.0
37345.0
54600.0
52962.0
96000.0
93120.0
143400.0
139098.0
43600.0
42292.0
136600.0
132502.0
46200.0
44814.0
324600.0
314862.0
27100.0
26287.0
31900.0
30943.0
40600.0
39382.0
40900.0
39673.0
67900.0
65863.0
93000.0
90210.0
28500.0
27645.0
57500.0
55775.0
148400.0
143948.0
24500.0
23765.0
76700.0
74399.0
54400.0
52768.0
30900.0
29973.0
38500.0
37345.0
52500.0
50925.0
27900.0
27063.0
72600.0
70422.0
39300.0
38121.0
76000.0
73720.0
204700.0
198559.0
23300.0
22601.0
51500.0
49955.0
40900.0
39673.0
34400.0
33368.0
5700.0
5529.0
29000.0
28130.0
76700.0
74399.0
54500.0
52865.0
64300.0
62371.0
25200.0
24444.0
68600.0
66542.0
52800.0
51216.0
33100.0
32107.0
52300.0
50731.0
48600.0
47142.0
37300.0
36181.0
36800.0
35696.0
38000.0
36860.0
20700.0
20079.0
45500.0
44135.0
93900.0
91083.0
22900.0
22213.0
84900.0
82353.0
144700.0
140359.0
44100.0
42777.0
86800.0
8419

In [ ]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,4,21,3,2014-01-25,102700.0
1,2,11,22,3,2014-01-25,41900.0
2,3,3,23,3,2014-01-28,91400.0
3,4,18,24,2,2014-01-28,45400.0
4,5,12,25,3,2014-01-29,70500.0
...,...,...,...,...,...,...
937,938,817,499,2,2020-12-22,44600.0
938,939,964,1518,3,2020-12-22,45600.0
939,940,962,1519,2,2020-12-23,42600.0
940,941,965,1520,2,2020-12-24,62500.0


In [ ]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

np.int64(0)

In [ ]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [ ]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(np.int64(33), array([35, 47, 46])), (np.int64(42), array([ 5, 39])), (np.int64(44), array([32])), (np.int64(50), array([38, 17])), (np.int64(52), array([42])), (np.int64(61), array([25, 35])), (np.int64(69), array([5, 0])), (np.int64(74), array([44, 60])), (np.int64(75), array([53])), (np.int64(85), array([9])), (np.int64(90), array([54])), (np.int64(106), array([48])), (np.int64(134), array([56,  1])), (np.int64(158), array([37, 53])), (np.int64(160), array([3])), (np.int64(171), array([29])), (np.int64(177), array([45])), (np.int64(190), array([9])), (np.int64(191), array([56])), (np.int64(212), array([40])), (np.int64(218), array([12])), (np.int64(221), array([56, 59])), (np.int64(224), array([18,  9])), (np.int64(234), array([51])), (np.int64(245), array([29])), (np.int64(254), array([41])), (np.int64(306), array([35])), (np.int64(275), array([62])), (np.int64(308), array([34])), (np.int64(319), array([32, 34, 33])), (np.int64(327), array([58])), (np.int64(340), array([38])), (np

In [ ]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [ ]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Przewód hamulcowy,,50,1,1
1,Zawór EGR,,500,1,1
2,Tłumik,,200,1,1
3,Opona zimowa,R16,65,2,4
4,Silnik,Hero Honda Passion Plus 0.4,737,2,1
...,...,...,...,...,...
167,Lusterko zewnętrzne lewe,Hyundai Ioniq szary,25,112,1
168,Rozrusznik,Volkswagen Golf Sportsvan,612,113,1
169,Drzwi prawe przednie,Volkswagen Golf Sportsvan czerwony,701,113,1
170,Dystrybutor,Suzuki Splash,174,114,1


# Wyposażenie

In [ ]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [ ]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Przewód hamulcowy,,50.0
1,2,Zawór EGR,,500.0
2,3,Tłumik,,200.0
3,4,Opona zimowa,R16,65.0
4,5,Silnik,Hero Honda Passion Plus 0.4,737.0
...,...,...,...,...
167,145,Lusterko zewnętrzne lewe,Hyundai Ioniq szary,25.0
168,146,Rozrusznik,Volkswagen Golf Sportsvan,612.0
169,147,Drzwi prawe przednie,Volkswagen Golf Sportsvan czerwony,701.0
170,148,Dystrybutor,Suzuki Splash,174.0


In [ ]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)

# Usługi - uzupełnienie

In [ ]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.1, 1.4), -1).astype(int)

In [ ]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,33,65,4,2014-03-11,910
1,2,42,80,6,2014-03-30,1210
2,3,44,84,6,2014-04-05,60
3,4,50,93,4,2014-04-19,250
4,5,52,98,4,2014-04-27,400
...,...,...,...,...,...,...
110,111,808,1326,7,2020-07-23,2240
111,112,943,1495,7,2020-10-06,30
112,113,948,195,4,2020-10-20,1590
113,114,960,1511,7,2020-12-06,210


In [ ]:
df_services.to_csv("outputs/usługi.csv", index=False)

# Użyte części - uzupełnienie

In [ ]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość']]

In [ ]:
used_parts_df

,id,id_usługi,id_części,ilość
0,1,1,1,1
1,2,1,2,1
2,3,1,3,1
3,4,2,4,4
4,5,2,5,1
...,...,...,...,...
167,168,112,145,1
168,169,113,146,1
169,170,113,147,1
170,171,114,148,1


In [ ]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [ ]:
def get_service_date(part_id):
    
    index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
    service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
    return service_date
    

buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'ilość'])

for i in range(len(buy_parts_quantity)):
    part_id = buy_parts_quantity['id_części'][i]
    quantity = buy_parts_quantity['ilość'][i]
    price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
    if price > 1000:
        quantity += rng.random() < 0.5
    elif price > 500:
        quantity += rng.choice([0, 1, 2], p=[0.2, 0.5, 0.3])
    else:
        quantity += rng.binomial(25, 0.15)
    total_cost = np.floor(quantity * price * rng.uniform(0.75, 0.95))[0]
    service_date = get_service_date(part_id)
    
    row = [int(i+1), int(part_id), total_cost, int(quantity)]
    parts_purchase_pd.loc[i] = row


In [ ]:
parts_purchase_pd

,id,id_części,koszt,ilość
0,1.0,1.0,469.0,12.0
1,2.0,2.0,3325.0,8.0
2,3.0,3.0,613.0,4.0
3,4.0,4.0,892.0,18.0
4,5.0,5.0,1233.0,2.0
...,...,...,...,...
144,145.0,145.0,45.0,2.0
145,146.0,146.0,1565.0,3.0
146,147.0,147.0,1628.0,3.0
147,148.0,148.0,663.0,5.0
